In [58]:
import numpy as np
import copy

In [13]:
def softmax(z):
    #z: (N,C)
    e_z = np.exp(z)
    A = e_z/e_z.sum(axis=1, keepdims = True)
    return A

In [7]:
def softmax_stable(z):
    e_z = np.exp(z - np.max(z, axis=1, keepdims=True))
    A = e_z /e_z.sum(axis=1, keepdims=True)
    return A

In [62]:
def compute_cost(X,y,w):
    '''
    X: (N,d)
    y: (N,)
    w: (d,C)
        A = softmax_stable(np.dot(X,w))
        m,n = A.shape
        J=0.
        for i in range(m):
            for j in range(n):
                if j==y[i]:
                    J+=np.log(A[i,j])
        
        J = -J/m
        return J 
    '''
    A = softmax_stable(X.dot(w))
    id0 = range(X.shape[0]) 
    return-np.mean(np.log(A[id0, y]))

In [63]:
def softmax_gradient(X,y,w):
    '''
    w: (d,C)
    X: (N,d)
    y: (N,)
    '''
    A = softmax_stable(X.dot(w)) # (N,C)
    id0 = range(A.shape[0])
    A[id0,y] -=1 # mỗi đầu ra đúng thì trừ đi 1: err
    return X.T.dot(A)/X.shape[0]  # 1/N *X* E.T : E chính là A

In [64]:
def softmax_fit(X,y,w_in,lr=0.01, nepochs=100, tol=1e-5, batch_size=10):
    w=w_old = w_in.copy()
    it=0
    J_hist = [compute_cost(X,y,w)]
    N=X.shape[0]
    nbatches = int(np.ceil(float(N)/batch_size)) 
    #tính số lượng batch cần thiết để lặp qua toàn bộ dữ liệu
    
    while it < nepochs:
        it+=1
        mix_ids = np.random.permutation(N)
        for i in range(nbatches):
            # lấy chỉ số dữ liệu trong batch hiện tại
            batch_ids = mix_ids[batch_size*i : min(batch_size*(i+1), N)]
            X_batch, y_batch = X[batch_ids], y[batch_ids]
            w-= lr * softmax_gradient(X_batch,y_batch,w)
            J_hist.append(compute_cost(X,y,w))
        if np.linalg.norm(w_old - w) < tol:
            break
        w_old = w.copy()
    return w, J_hist
            

In [65]:
def predict(w,X):
    return np.argmax(X.dot(w), axis=1)

In [90]:
C, N = 5, 500 # number of classes and number of points per class
means = [[2, 2], [8, 3], [3, 6], [14, 2], [12, 8]]
cov = [[1, 0], [0, 1]]
X0 = np.random.multivariate_normal(means[0], cov, N)
X1 = np.random.multivariate_normal(means[1], cov, N)
X2 = np.random.multivariate_normal(means[2], cov, N)
X3 = np.random.multivariate_normal(means[3], cov, N)
X4 = np.random.multivariate_normal(means[4], cov, N)
X = np.concatenate((X0, X1, X2, X3, X4), axis = 0) # each row is a datapoint
Xbar = np.concatenate((X, np.ones((X.shape[0], 1))), axis = 1) # bias trick
y = np.asarray([0]*N + [1]*N + [2]*N+ [3]*N + [4]*N) # label
W_init = np.random.randn(Xbar.shape[1], C)
W, loss_hist = softmax_fit(Xbar, y, W_init, lr = 0.05)

y_pred = predict(W,Xbar)

In [93]:
count = y==y_pred
print(sum(count)/len(y))

0.676
